In [14]:
%load_ext autoreload
%autoreload 2

from time import time
import pandas as pd
import numpy as np
import os
from collections import Counter, defaultdict
import pickle
import sys
sys.path.insert(0, "/data3/muntean/DRhard")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load qrels

In [8]:
# GET qrels
qrel_path='/data3/muntean/conversational-cache/data/CAST_qrels/qrels-docs.2019.txt'
qrels_df = pd.read_csv(qrel_path, delimiter=" ", header=None)

# rilevanti
qrels_df = qrels_df[qrels_df[3]>0]

# lista rilevanti
rel_docs_list  = list(qrels_df[2])
len(rel_docs_list)

,0,1,2,3
1,31_1,0,CAR_1463f964653c5c9f614a0a88d26b175e4a8120f1,1
2,31_1,0,CAR_172e16e89ea3d5546e53384a27c3be299bcfe968,2
4,31_1,0,CAR_2174ad0aa50712ff24035c23f59a3c2b43267650,3
5,31_1,0,CAR_25a576af9caa6422f55c2acf945dc79b423fb41e,1
6,31_1,0,CAR_2dc597ac2fc10917a752552bc335e6ac1aedc3f0,2
...,...,...,...,...
29335,79_9,0,MARCO_6800093,2
29337,79_9,0,MARCO_7647252,2
29342,79_9,0,MARCO_8455238,2
29343,79_9,0,MARCO_8458145,2


In [39]:
with open('/data3/muntean/DRhard/data/passage/relevant_docs_ids.txt', 'w') as filehandle:
    for index, row in qrels_df.iterrows():
        filehandle.write("{}\t{}\n".format(row[0], row[2]))

# Load embeddings

In [9]:
doc_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/passages.memmap"
docid_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/passages-id.memmap"

In [10]:
doc_embeddings = np.memmap(doc_memmap_path, dtype=np.float32, mode="r")
doc_ids = np.memmap(docid_memmap_path, dtype=np.int32, mode="r")
doc_embeddings = doc_embeddings.reshape(-1, 768)

# Load id maps

In [11]:
# collection id dict
collection_mapping_path = "/data3/muntean/DRhard/data/passage/dataset/CASTcollectionPID2newID.tsv"
collection_df = pd.read_csv(collection_mapping_path, delimiter="\t", header=None)
print(len(collection_df))

38626614


In [12]:
pid2newpid_dict = dict(zip(collection_df[0], collection_df[1])) 
newpid2pid_dict = dict(zip(collection_df[1], collection_df[0])) 

In [15]:
# DRhard docid and qid encoding
preprocess_dir = "/data3/muntean/DRhard/data/passage/preprocess"

pid2offset = pickle.load(open(os.path.join(preprocess_dir, "pid2offset.pickle"), 'rb'))
offset2pid = {v:k for k, v in pid2offset.items()}

# Convert passage ids to star ids

In [17]:
pid2newpid_rel_doc_list = [pid2newpid_dict[x] for x in rel_docs_list]

In [18]:
newpid2starid = [pid2offset[x] for x in pid2newpid_rel_doc_list]

In [19]:
relevant_embeddings = doc_embeddings[newpid2starid]

In [22]:
relevant_embeddings.shape

(8120, 768)

# Save embeddings to file

In [21]:
relevant_embeddings.tofile("/data3/muntean/DRhard/data/passage/relevant_docs_embeddings.memmap")

# Check for normalization

In [24]:
relevant_embeddings[0]

array([-8.96443427e-01,  4.42078769e-01, -5.16832948e-01,  2.35813469e-01,
       -3.59534979e-01, -3.44220251e-01,  1.54462278e+00, -4.19642895e-01,
        1.80079210e+00,  3.96456957e-01,  3.85502458e-01,  1.35248137e+00,
        4.04327124e-01, -1.01331592e+00, -2.34779999e-01, -1.81400001e+00,
        4.89553452e-01, -1.39943846e-02,  1.95318498e-02, -5.56835294e-01,
       -6.49499953e-01,  1.03930986e+00, -7.48351812e-01, -5.27582824e-01,
        5.54653704e-01, -6.20283306e-01, -9.07481089e-03, -7.23088861e-01,
        9.46863115e-01,  7.43842781e-01, -1.25214294e-01, -1.08732772e+00,
        2.81023264e-01,  5.16518533e-01,  1.34593379e+00, -1.58148718e+00,
        6.53436244e-01, -1.51687920e+00,  1.89995646e-01,  4.57807690e-01,
        3.94304156e-01,  5.96680224e-01, -1.19116354e+00,  1.28795981e-01,
        9.74635124e-01,  1.49428380e+00, -1.89887857e+00, -1.46228266e+00,
       -3.36609215e-01,  1.27887160e-01, -2.15675011e-01,  1.47385812e+00,
       -1.18487060e+00, -

In [26]:
np.sqrt(sum([x*x for x in relevant_embeddings[0]]))

24.84152585963421